# Good Review Srcapper

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

In [2]:
links = pd.read_excel('dataset.xlsx')

In [57]:
amazon_links = links['Amazon link']

In [58]:
titles=[]

In [59]:
for i in range (len(amazon_links)):
    url=amazon_links[i]
    title=url.split("/dp/")[0].split('.com/')[1]
    titles.append(title)

In [3]:
goodread_links = links['Goodreads link']

In [4]:
goodread_links.head()

0    https://www.goodreads.com/book/show/54976983-f...
1    https://www.goodreads.com/book/show/53370054-f...
2    https://www.goodreads.com/book/show/45213808-t...
3    https://www.goodreads.com/book/show/53258232-t...
4    https://www.goodreads.com/book/show/45452530-r...
Name: Goodreads link, dtype: object

In [61]:
def reviews(base_url):
    
    book_reviews = []
    ratings=[]
    dates=[]
    
    HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
    
    URL = base_url
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "lxml")
    reviews=soup.find_all('span',style="display:none")
    book_ratings = soup.find_all('span',attrs={'class':'staticStars notranslate'})
    date = soup.find_all('a',attrs={'class':'reviewDate createdAt right'})
    
    for i in range(1,len(reviews)):
        book_reviews.append(reviews[i].text)
    
    for i in range(1,len(book_ratings)):
        ratings.append(book_ratings[i].text)
        
    for i in range (len(ratings)):
        if ratings[i]=='it was amazing':
            ratings[i]=5
        elif ratings[i]=='really liked it':
            ratings[i]=4
        elif ratings[i]=='liked it':
            ratings[i]=3
        elif ratings[i]=='it was ok':
            ratings[i]=2
        elif ratings[i]=='did not like it':
            ratings[i]=1
        else:
            ratings[i]=' '
   
        
    for i in range(1,len(date)):
        dates.append(date[i].text)
    
    for j in range(len(dates)):
        dates[j]=datetime.strptime(dates[j],'%b %d, %Y').strftime('%Y-%m-%d')
    
        
    df=pd.DataFrame(list(zip(book_reviews,ratings,dates)),columns =['Review', 'Rating','Date'])
    return df

In [62]:
goodread_reviews={}

In [64]:
length=len(goodread_links)

In [65]:
for i in range (length):
    df = reviews(goodread_links[i])
    d={titles[i]:df}
    goodread_reviews.update(d)

In [66]:
reviews=pd.concat(goodread_reviews)

In [67]:
reviews['source']='GoodRead'

In [69]:
reviews.head()

Review  \
Finding-Mother-Tree-Discovering-Wisdom 0  In Finding the Mother Tree a world-leading exp...   
                                       1  So full disclosure - I discovered Suzanne Sima...   
                                       2  In brief - In the strange world we live in thi...   
                                       3  \ni can't tell if my blood is in the trees or ...   
                                       4  As an avid gardener most of my life I'm very f...   

                                         Rating        Date    source  
Finding-Mother-Tree-Discovering-Wisdom 0      5  2021-03-04  GoodRead  
                                       1      4  2021-05-09  GoodRead  
                                       2      4  2021-03-07  GoodRead  
                                       3      3  2021-05-14  GoodRead  
                                       4      4  2021-03-26  GoodRead

In [68]:
reviews.to_csv('goodread_review.csv',index=False)